# NLP CW Task 1

  - Approach 1, which can use use pre-trained embeddings / models
  - Approach 2, which should not use any pre-trained embeddings or models

#### Running your code:
  - Your models should run automatically when running your colab file without further intervention
  - For each task you should automatically output the performance of both models
  - Your code should automatically download any libraries required

#### Structure of your code:
  - You are expected to use the 'train', 'eval' and 'model_performance' functions, although you may edit these as required
  - Otherwise there are no restrictions on what you can do in your code

#### Documentation:
  - You are expected to produce a .README file summarising how you have approached both tasks
  - Your .README file should explain how to replicate the different experiments mentioned in your report

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, random_split
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import MultinomialNB
import codecs
import tqdm
import re

In [ ]:
# Get word embeddings
if not os.path.exists('glove.6B.50d.txt'):
  !wget -q --show-progress http://nlp.stanford.edu/data/glove.6B.zip -O glove.zip
  !unzip -q glove.zip

# Install packages that Colab does not provide automatically
!pip -q install transformers
from transformers import RobertaTokenizer, RobertaModel, BertPreTrainedModel, DistilBertModel, DistilBertTokenizer

# Get test and train data files
if not os.path.exists('dev.csv'):
  !wget -q --show-progress https://raw.githubusercontent.com/matt-malarkey/nlp-cw-data/master/dev.csv
  !wget -q --show-progress https://raw.githubusercontent.com/matt-malarkey/nlp-cw-data/master/train.csv

In [ ]:
# Setting random seed and device
SEED = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
# Load data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('dev.csv')

In [ ]:
# Number of epochs
epochs = 10

# Proportion of training data for train compared to dev
train_proportion = 0.8

In [ ]:
# We define our training loop
def train(train_iter, dev_iter, model, number_epoch):
  """
  Training loop for the model, which calls on eval to evaluate after each epoch
  """

  print("Training model.")

  loss_fn = nn.MSELoss()
  loss_fn = loss_fn.to(device)

  for epoch in range(1, number_epoch+1):
    model.train()

    epoch_loss = 0
    epoch_sse = 0
    no_observations = 0  # Observations used for training so far

    for batch in train_iter:
      ids = batch['ids'].to(device).squeeze()
      mask = batch['mask'].to(device).squeeze()
      target = batch['target'].to(device, dtype=torch.float)
      extra_features = batch['extra_features'].to(device)

      predictions = model(ids, mask, extra_features).squeeze(1)
      optimizer.zero_grad()
      loss = loss_fn(predictions, target)

      no_observations = no_observations + target.shape[0]
      sse, __ = model_performance(predictions.detach().cpu().numpy(), target.detach().cpu().numpy())

      loss.backward()
      optimizer.step()

      epoch_loss += loss.item()*target.shape[0]
      epoch_sse += sse

    valid_loss, valid_mse, __, __ = eval(dev_iter, model)

    epoch_loss, epoch_mse = epoch_loss / no_observations, epoch_sse / no_observations
    print(f'| Epoch: {epoch:02} | Train Loss: {epoch_loss:.2f} | Train MSE: {epoch_mse:.2f} | Train RMSE: {epoch_mse**0.5:.2f} | \
    Val. Loss: {valid_loss:.2f} | Val. MSE: {valid_mse:.2f} |  Val. RMSE: {valid_mse**0.5:.2f} |')

In [ ]:
# We evaluate performance on our dev set
def eval(data_iter, model):
  """
  Evaluating model performance on the dev set
  """
  model.eval()
  epoch_loss = 0
  epoch_sse = 0
  pred_all = []
  trg_all = []
  no_observations = 0
  loss_fn = nn.MSELoss()
  loss_fn = loss_fn.to(device)

  with torch.no_grad():
    for batch in data_iter:
      ids = batch['ids'].to(device).squeeze()
      mask = batch['mask'].to(device).squeeze()
      target = batch['target'].to(device, dtype=torch.float)
      extra_features = batch['extra_features'].to(device)

      predictions = model(ids, mask, extra_features).squeeze(1)

      loss = loss_fn(predictions, target)

      # We get the mse
      pred, trg = predictions.detach().cpu().numpy(), target.detach().cpu().numpy()
      sse, __ = model_performance(pred, trg)

      no_observations = no_observations + target.shape[0]
      epoch_loss += loss.item()*target.shape[0]
      epoch_sse += sse
      pred_all.extend(pred)
      trg_all.extend(trg)

  return epoch_loss/no_observations, epoch_sse/no_observations, np.array(pred_all), np.array(trg_all)

In [ ]:
# How we print the model performance
def model_performance(output, target, print_output=False):
  """
  Returns SSE and MSE per batch (printing the MSE and the RMSE)
  """
  sq_error = (output - target)**2

  sse = np.sum(sq_error)
  mse = np.mean(sq_error)
  rmse = np.sqrt(mse)

  if print_output:
    print(f'| MSE: {mse:.2f} | RMSE: {rmse:.2f} |')

  return sse, mse

## Preprocessing

In [ ]:
# Pattern to match words and multiword tags
# See https://regex101.com/r/a2cWZL/1 for example
# TOKEN_PATTERN = re.compile("[A-Za-z#]+|<[A-Za-z#]+\s[A-Za-z]+\/>")

# Patten to match only words
TOKEN_PATTERN = re.compile("\w+")

# TODO: processing of:
# - hashtags,
# - words with a "-" in between them
# - punctuation
# - entity names

def tokenize(sentence):
  return TOKEN_PATTERN.findall(sentence)

# Remove angle brackets and lowercase
def preprocess(word):
  if len(word) > 2 and word[0] == '<' and word[-2] == '/' and word[-1] == '>' :
    word = word[1:-2]
  return word.lower()

def create_vocab(data):
  """
  Creating a corpus of all the tokens used
  """
  # Let us put the tokenized corpus in a list
  tokenized_corpus = [[pre(t) for t in tokenize(s)] for s in data]

  # Create single list of all vocabulary
  vocabulary = []
  for sentence in tokenized_corpus:
    for token in sentence:
      if token not in vocabulary:
        vocabulary.append(token)

  return vocabulary, tokenized_corpus

# Extra features:
# [0] - binary if 'Trump' in headline
# TODO ...
def extract_features(headline):
  extra_features = [0]
  if 'Trump' in headline:
    extra_features[0] = 1
  return extra_features

In [ ]:
# Inherits from Dataset class so the DataLoader can use it
class Task1Dataset(Dataset):

  def __init__(self, tokenizer, train_data, labels):
    self.x_train = train_data
    self.y_train = labels
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.y_train)

  def __getitem__(self, item):
    # Preprocess headline before embedding
    headline = self.x_train[item]
    processed_headline = ' '.join([pre(t) for t in tokenize(headline)])

    target = self.y_train[item]
    encoding = self.tokenizer.encode_plus(processed_headline, return_tensors='pt',
                                          padding='max_length', truncation=True,
                                          max_length=128, pad_to_max_length=True)
    ids = encoding['input_ids']
    mask = encoding['attention_mask']

    extra_features = extract_features(headline)

    return {
      'ids': torch.tensor(ids),
      'mask': torch.tensor(mask),
      'target': torch.tensor(target),
      'extra_features': torch.tensor(extra_features)
    }

In [ ]:
class DistilBertRegressionModel(nn.Module):
  def __init__(self):
    super(DistilBertRegressionModel, self).__init__()

    self.bert = DistilBertModel.from_pretrained('distilbert-base-uncased')
    self.num_extra_features = 1
    self.linear1 = nn.Linear(768, 32)
    self.linear2 = nn.Linear(32 + self.num_extra_features, 1)
    self.loss = nn.MSELoss()

  def forward(self, ids, mask, extra_features):
    embeddings = self.bert(input_ids=ids, attention_mask=mask)[0]
    x = self.linear1(embeddings[:, 0])
    concat_features = torch.cat([x, extra_features], dim=1)
    x = self.linear2(concat_features)
    return x

### Approach 1: Usiung pre-trained representations

In [ ]:
BATCH_SIZE = 32

# Initialise RoBERTa model and tokenizer
model = DistilBertRegressionModel().to(device)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Set training and test data
training_data = train_df['original']
training_labels = train_df['meanGrade']
test_data = test_df['original']
train_and_dev = Task1Dataset(tokenizer, training_data, training_labels)

# Split training data into train and dev sets
train_examples = round(len(train_and_dev)*train_proportion)
dev_examples = len(train_and_dev) - train_examples
train_dataset, dev_dataset = random_split(train_and_dev, (train_examples, dev_examples))

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE)
dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=BATCH_SIZE)

In [ ]:
# Do the training
optimizer = torch.optim.Adam(model.parameters())
train(train_loader, dev_loader, model, epochs)

### Approach 2: No pre-trained representations

In [ ]:
train_and_dev = train_df['edit']

training_data, dev_data, training_y, dev_y = train_test_split(train_df['edit'], train_df['meanGrade'],
                                                                        test_size=(1-train_proportion),
                                                                        random_state=42)

# We train a Tf-idf model
count_vect = CountVectorizer(stop_words='english')
train_counts = count_vect.fit_transform(training_data)
transformer = TfidfTransformer().fit(train_counts)
train_counts = transformer.transform(train_counts)
regression_model = LinearRegression().fit(train_counts, training_y)

# Train predictions
predicted_train = regression_model.predict(train_counts)

# Calculate Tf-idf using train and dev, and validate model on dev:
test_and_test_counts = count_vect.transform(train_and_dev)
transformer = TfidfTransformer().fit(test_and_test_counts)

test_counts = count_vect.transform(dev_data)

test_counts = transformer.transform(test_counts)

# Dev predictions
predicted = regression_model.predict(test_counts)

# We run the evaluation:
print("\nTrain performance:")
sse, mse = model_performance(predicted_train, training_y, True)

print("\nDev performance:")
sse, mse = model_performance(predicted, dev_y, True)

#### Baseline for task 2

In [ ]:
# Baseline for the task
pred_baseline = torch.zeros(len(dev_y)) + np.mean(training_y)
print("\nBaseline performance:")
sse, mse = model_performance(pred_baseline, dev_y, True)